In [1]:
import os
import json
ROOT_DIR = os.getcwd()
document_type = "bank_statements"
template_name = "halifax"
identifier = "april"

pdf_path: str = os.path.join(
    ROOT_DIR,
    "data",
    document_type,
    template_name,
    "pdf",
    f"{template_name}_{identifier}.pdf",
)

In [2]:
output_path: str = os.path.join(
    ROOT_DIR,
    "outputs",
    f"{template_name}_{identifier}_output.json",
)


# output = json.load(open(output_path))

In [3]:
output = {
    "metadata": {
        "document_id": "7895ae19-ac9a-424d-81be-6c68c06ebc83",
        "parsed_at": "2024-12-12T15:46:37",
        "number_of_pages": 2
    },
    "pages": [
        {
            "forms": [
                {
                    "money_in": "\u00a3761.80"
                },
                {
                    "start_balance": "\u00a3521.23"
                },
                {
                    "money_out": "\u00a3777.93"
                },
                {
                    "end_balance": "\u00a3505.10"
                }
            ],
            "tables": [
                {
                    "data": [
                        {
                            "date": "03 Apr 23",
                            "type": "DD",
                            "description": "UNIVERSITY OF NOTT",
                            "money_out": "22.90",
                        },
                        {
                            "date": "06 Apr 23",
                            "type": "FPI",
                            "description": "HIYACAR LTD",
                            "money_out": "",
                        },
                    ]
                }
            ]
        }
    ]
}

In [4]:
from cleaner import DocumentCleaner

if __name__ == "__main__":
    path = os.path.join(ROOT_DIR, "cleaner_config", "halifax_cleaner_config.json")
    cleaner = DocumentCleaner(path)

    for page in output["pages"]:
        cleaned_document = cleaner.clean_page(page)
        print(cleaned_document)



{'forms': [{'money_in': '761.80'}, {'start_balance': '521.23'}, {'money_out': '777.93'}, {'end_balance': '505.10'}], 'tables': [{'data': [{'date': '2023-04-03', 'type': 'DD', 'description': 'UNIVERSITY OF NOTT', 'money_out': '22.90'}, {'date': '2023-04-06', 'type': 'FPI', 'description': 'HIYACAR LTD', 'money_out': ''}]}]}


In [5]:
page

{'forms': [{'money_in': '£761.80'},
  {'start_balance': '£521.23'},
  {'money_out': '£777.93'},
  {'end_balance': '£505.10'}],
 'tables': [{'data': [{'date': '03 Apr 23',
     'type': 'DD',
     'description': 'UNIVERSITY OF NOTT',
     'money_out': '22.90'},
    {'date': '06 Apr 23',
     'type': 'FPI',
     'description': 'HIYACAR LTD',
     'money_out': ''}]}]}

In [6]:
cleaned_document

{'forms': [{'money_in': '761.80'},
  {'start_balance': '521.23'},
  {'money_out': '777.93'},
  {'end_balance': '505.10'}],
 'tables': [{'data': [{'date': '2023-04-03',
     'type': 'DD',
     'description': 'UNIVERSITY OF NOTT',
     'money_out': '22.90'},
    {'date': '2023-04-06',
     'type': 'FPI',
     'description': 'HIYACAR LTD',
     'money_out': ''}]}]}

In [12]:
import os
import json

outputs = []
for f in os.listdir('outputs'):
    file_path = os.path.join('outputs', f)
    if os.path.isfile(file_path) and f.endswith('.json'):
        with open(file_path, 'r') as json_file:
            outputs.append(json.load(json_file))

In [18]:
import pandas as pd

descriptions = []

for output in outputs:
    pages = output["pages"]
    for page in pages:
        tables = page["tables"]
        for table in tables:
            data = table["data"]
            for row in data:
                description = row["description"]
                descriptions.append(description.lower())

df = pd.DataFrame(descriptions, columns=["description"])


In [19]:
df

,description
0,trainline +443332022222 gbr
1,c little (faster payments) reference: monzo cash
2,zettle_*oxpro limited oxford gbr
3,trainline +443332022222 gbr
4,dice.fm +442080687457 gbr
...,...
852,transfer from sort code 20-44-51 account 13415...
853,transfer from sort code 20-44-51 account 13415...
854,barclays bank uk plc. authorised by 2 regulati...
855,transfer from sort code 20-44-51 account 13415...


In [20]:
category_mapping = pd.read_csv("category_mapping.csv")

In [21]:
df['category'] = None
df['subcategory'] = None

for index, row in df.iterrows():
    description = row['description']
    for _, category_row in category_mapping.iterrows():
        if category_row['keyword'] in description:
            df.at[index, 'category'] = category_row['category']
            df.at[index, 'subcategory'] = category_row['subcategory']
            break


In [26]:
df.dropna()

,description,category,subcategory
82,hiyacar limited f salary,INCOME,salary
282,hiyacar limited f salary,INCOME,salary
815,hiyacar limited f salary,INCOME,salary
